## Chatbot Interattivo Phi 3 Mini 4K Instruct con Whisper

### Introduzione:
Il Chatbot Interattivo Phi 3 Mini 4K Instruct è uno strumento che consente agli utenti di interagire con la demo Microsoft Phi 3 Mini 4K Instruct utilizzando input testuali o audio. Il chatbot può essere utilizzato per una varietà di attività, come traduzioni, aggiornamenti meteo e raccolta di informazioni generali.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Crea il tuo Huggingface Access Token](https://huggingface.co/settings/tokens)

Crea un nuovo token  
Fornisci un nuovo nome  
Seleziona i permessi di scrittura  
Copia il token e salvalo in un luogo sicuro  


Il seguente codice Python esegue due compiti principali: importare il modulo `os` e impostare una variabile d'ambiente.

1. Importazione del modulo `os`:
   - Il modulo `os` in Python fornisce un modo per interagire con il sistema operativo. Permette di svolgere varie operazioni legate al sistema operativo, come accedere alle variabili d'ambiente, lavorare con file e directory, ecc.
   - In questo codice, il modulo `os` viene importato utilizzando l'istruzione `import`. Questa istruzione rende disponibile la funzionalità del modulo `os` per l'uso nello script Python corrente.

2. Impostazione di una variabile d'ambiente:
   - Una variabile d'ambiente è un valore che può essere accessibile dai programmi in esecuzione sul sistema operativo. È un modo per memorizzare configurazioni o altre informazioni che possono essere utilizzate da più programmi.
   - In questo codice, una nuova variabile d'ambiente viene impostata utilizzando il dizionario `os.environ`. La chiave del dizionario è `'HF_TOKEN'`, e il valore viene assegnato dalla variabile `HUGGINGFACE_TOKEN`.
   - La variabile `HUGGINGFACE_TOKEN` è definita appena sopra questo frammento di codice ed è assegnata a una stringa `"hf_**************"` utilizzando la sintassi `#@param`. Questa sintassi è spesso utilizzata nei notebook Jupyter per permettere l'inserimento di dati da parte dell'utente e la configurazione dei parametri direttamente nell'interfaccia del notebook.
   - Impostando la variabile d'ambiente `'HF_TOKEN'`, essa può essere accessibile da altre parti del programma o da altri programmi in esecuzione sullo stesso sistema operativo.

In sintesi, questo codice importa il modulo `os` e imposta una variabile d'ambiente chiamata `'HF_TOKEN'` con il valore fornito nella variabile `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Questo frammento di codice definisce una funzione chiamata clear_output, utilizzata per cancellare l'output della cella corrente in Jupyter Notebook o IPython. Analizziamo il codice e comprendiamone la funzionalità:

La funzione clear_output accetta un parametro chiamato wait, che è un valore booleano. Di default, wait è impostato su False. Questo parametro determina se la funzione deve attendere che sia disponibile un nuovo output per sostituire quello esistente prima di cancellarlo.

La funzione stessa viene utilizzata per cancellare l'output della cella corrente. In Jupyter Notebook o IPython, quando una cella produce un output, come testo stampato o grafici, quell'output viene visualizzato sotto la cella. La funzione clear_output consente di cancellare quell'output.

L'implementazione della funzione non è fornita nel frammento di codice, come indicato dall'ellissi (...). L'ellissi rappresenta un segnaposto per il codice effettivo che esegue la cancellazione dell'output. L'implementazione della funzione potrebbe coinvolgere l'interazione con l'API di Jupyter Notebook o IPython per rimuovere l'output esistente dalla cella.

In sintesi, questa funzione offre un modo pratico per cancellare l'output della cella corrente in Jupyter Notebook o IPython, facilitando la gestione e l'aggiornamento dell'output visualizzato durante sessioni di codifica interattive.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Esegui la sintesi vocale (TTS) utilizzando il servizio Edge TTS. Analizziamo le implementazioni delle funzioni rilevanti una per una:

1. `calculate_rate_string(input_value)`: Questa funzione prende un valore di input e calcola la stringa di velocità per la voce TTS. Il valore di input rappresenta la velocità desiderata del discorso, dove un valore di 1 rappresenta la velocità normale. La funzione calcola la stringa di velocità sottraendo 1 dal valore di input, moltiplicandolo per 100 e determinando il segno in base al fatto che il valore di input sia maggiore o uguale a 1. La funzione restituisce la stringa di velocità nel formato "{segno}{velocità}".

2. `make_chunks(input_text, language)`: Questa funzione prende un testo di input e una lingua come parametri. Divide il testo di input in frammenti basandosi sulle regole specifiche della lingua. In questa implementazione, se la lingua è "English", la funzione divide il testo a ogni punto (".") e rimuove eventuali spazi iniziali o finali. Successivamente, aggiunge un punto a ogni frammento e restituisce l'elenco filtrato dei frammenti.

3. `tts_file_name(text)`: Questa funzione genera un nome file per il file audio TTS basandosi sul testo di input. Esegue diverse trasformazioni sul testo: rimuove un punto finale (se presente), converte il testo in minuscolo, elimina spazi iniziali e finali e sostituisce gli spazi con trattini bassi. Successivamente, tronca il testo a un massimo di 25 caratteri (se più lungo) o utilizza il testo completo se è vuoto. Infine, genera una stringa casuale utilizzando il modulo [`uuid`] e la combina con il testo troncato per creare il nome file nel formato "/content/edge_tts_voice/{testo_troncato}_{stringa_casuale}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Questa funzione unisce più file audio in un unico file audio. Prende un elenco di percorsi di file audio e un percorso di output come parametri. La funzione inizializza un oggetto vuoto `AudioSegment` chiamato [`merged_audio`]. Successivamente, itera attraverso ogni percorso di file audio, carica il file audio utilizzando il metodo `AudioSegment.from_file()` della libreria `pydub` e aggiunge il file audio corrente all'oggetto [`merged_audio`]. Infine, esporta l'audio unito nel percorso di output specificato nel formato MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Questa funzione esegue l'operazione TTS utilizzando il servizio Edge TTS. Prende un elenco di frammenti di testo, la velocità del discorso, il nome della voce e il percorso di salvataggio come parametri. Se il numero di frammenti è maggiore di 1, la funzione crea una directory per memorizzare i file audio dei singoli frammenti. Successivamente, itera attraverso ogni frammento, costruisce un comando Edge TTS utilizzando la funzione `calculate_rate_string()`, il nome della voce e il testo del frammento, ed esegue il comando utilizzando la funzione `os.system()`. Se l'esecuzione del comando ha successo, aggiunge il percorso del file audio generato a un elenco. Dopo aver elaborato tutti i frammenti, unisce i file audio individuali utilizzando la funzione `merge_audio_files()` e salva l'audio unito nel percorso di salvataggio specificato. Se c'è solo un frammento, genera direttamente il comando Edge TTS e salva l'audio nel percorso di salvataggio. Infine, restituisce il percorso di salvataggio del file audio generato.

6. `random_audio_name_generate()`: Questa funzione genera un nome file audio casuale utilizzando il modulo [`uuid`]. Genera un UUID casuale, lo converte in una stringa, prende i primi 8 caratteri, aggiunge l'estensione ".mp3" e restituisce il nome file audio casuale.

7. `talk(input_text)`: Questa funzione è il punto di ingresso principale per eseguire l'operazione TTS. Prende un testo di input come parametro. Controlla prima la lunghezza del testo di input per determinare se si tratta di una frase lunga (maggiore o uguale a 600 caratteri). In base alla lunghezza e al valore della variabile `translate_text_flag`, determina la lingua e genera l'elenco dei frammenti di testo utilizzando la funzione `make_chunks()`. Successivamente, genera un percorso di salvataggio per il file audio utilizzando la funzione `random_audio_name_generate()`. Infine, chiama la funzione `edge_free_tts()` per eseguire l'operazione TTS e restituisce il percorso di salvataggio del file audio generato.

Nel complesso, queste funzioni lavorano insieme per dividere il testo di input in frammenti, generare un nome file per il file audio, eseguire l'operazione TTS utilizzando il servizio Edge TTS e unire i file audio individuali in un unico file audio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

L'implementazione di due funzioni: convert_to_text e run_text_prompt, così come la dichiarazione di due classi: str e Audio.

La funzione convert_to_text prende un audio_path come input e trascrive l'audio in testo utilizzando un modello chiamato whisper_model. La funzione verifica innanzitutto se il flag gpu è impostato su True. Se lo è, il whisper_model viene utilizzato con determinati parametri come word_timestamps=True, fp16=True, language='English' e task='translate'. Se il flag gpu è False, il whisper_model viene utilizzato con fp16=False. La trascrizione risultante viene quindi salvata in un file chiamato 'scan.txt' e restituita come testo.

La funzione run_text_prompt prende un messaggio e una chat_history come input. Utilizza la funzione phi_demo per generare una risposta da un chatbot basata sul messaggio di input. La risposta generata viene poi passata alla funzione talk, che converte la risposta in un file audio e restituisce il percorso del file. La classe Audio viene utilizzata per visualizzare e riprodurre il file audio. L'audio viene visualizzato utilizzando la funzione display del modulo IPython.display, e l'oggetto Audio viene creato con il parametro autoplay=True, in modo che l'audio inizi a suonare automaticamente. La chat_history viene aggiornata con il messaggio di input e la risposta generata, e vengono restituiti una stringa vuota e la chat_history aggiornata.

La classe str è una classe integrata in Python che rappresenta una sequenza di caratteri. Fornisce vari metodi per manipolare e lavorare con le stringhe, come capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill e altri. Questi metodi permettono di eseguire operazioni come ricerca, sostituzione, formattazione e manipolazione delle stringhe.

La classe Audio è una classe personalizzata che rappresenta un oggetto audio. Viene utilizzata per creare un lettore audio nell'ambiente Jupyter Notebook. La classe accetta vari parametri come data, filename, url, embed, rate, autoplay e normalize. Il parametro data può essere un array numpy, una lista di campioni, una stringa che rappresenta un nome di file o URL, o dati PCM grezzi. Il parametro filename viene utilizzato per specificare un file locale da cui caricare i dati audio, e il parametro url viene utilizzato per specificare un URL da cui scaricare i dati audio. Il parametro embed determina se i dati audio devono essere incorporati utilizzando un URI di dati o referenziati dalla fonte originale. Il parametro rate specifica la frequenza di campionamento dei dati audio. Il parametro autoplay determina se l'audio deve iniziare a suonare automaticamente. Il parametro normalize specifica se i dati audio devono essere normalizzati (riscalati) al massimo intervallo possibile. La classe Audio fornisce anche metodi come reload per ricaricare i dati audio da file o URL, e attributi come src_attr, autoplay_attr e element_id_attr per recuperare gli attributi corrispondenti per l'elemento audio in HTML.

In generale, queste funzioni e classi vengono utilizzate per trascrivere audio in testo, generare risposte audio da un chatbot e visualizzare e riprodurre audio nell'ambiente Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
